## Deliverable 2. Create a Customer Travel Destinations Map.

In [10]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [11]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,East London,ZA,-33.0153,27.9116,52.74,87,75,11.50,moderate rain
1,1,Puerto Ayora,EC,-0.7393,-90.3518,75.16,80,67,7.00,broken clouds
2,2,Bowen,AU,-20.0167,148.2333,62.82,88,49,10.33,scattered clouds
3,3,Sabha,LY,27.0377,14.4283,104.65,7,2,13.22,clear sky
4,4,Eenhana,NaN,-17.4667,16.3333,85.75,13,0,6.20,clear sky


In [14]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_temp = float(input('What is the minimum temperature you would like for your trip? '))

What is the minimum temperature you would like for your trip? 75
What is the minimum temperature you would like for your trip? 90


In [22]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp']>= min_temp) & (city_data_df['Max Temp']<= max_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Puerto Ayora,EC,-0.7393,-90.3518,75.16,80,67,7.00,broken clouds
4,4,Eenhana,NaN,-17.4667,16.3333,85.75,13,0,6.20,clear sky
6,6,San Cristobal,VE,7.7669,-72.2250,82.58,46,31,7.83,scattered clouds
12,12,Matamoros,MX,25.5333,-103.2500,83.10,37,87,3.29,overcast clouds
16,16,Wajid,SO,3.8093,43.2471,86.72,35,99,15.82,overcast clouds
23,23,Kutum,SD,14.2000,24.6667,78.04,57,78,3.91,broken clouds
28,28,Bengkulu,ID,-3.8004,102.2655,75.47,90,91,3.85,overcast clouds
30,30,Ewa Beach,US,21.3156,-158.0072,76.95,72,1,3.44,clear sky
31,31,Pontianak,ID,-0.0333,109.3333,75.52,91,100,4.27,overcast clouds
34,34,Cassilandia,BR,-19.1133,-51.7342,89.85,20,3,5.01,clear sky


In [23]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                244
City                   244
Country                243
Lat                    244
Lng                    244
Max Temp               244
Humidity               244
Cloudiness             244
Wind Speed             244
Current Description    244
dtype: int64

In [27]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Puerto Ayora,EC,-0.7393,-90.3518,75.16,80,67,7.00,broken clouds
6,6,San Cristobal,VE,7.7669,-72.2250,82.58,46,31,7.83,scattered clouds
12,12,Matamoros,MX,25.5333,-103.2500,83.10,37,87,3.29,overcast clouds
16,16,Wajid,SO,3.8093,43.2471,86.72,35,99,15.82,overcast clouds
23,23,Kutum,SD,14.2000,24.6667,78.04,57,78,3.91,broken clouds


In [76]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Puerto Ayora,EC,75.16,broken clouds,-0.7393,-90.3518,
6,San Cristobal,VE,82.58,scattered clouds,7.7669,-72.2250,
12,Matamoros,MX,83.10,overcast clouds,25.5333,-103.2500,
16,Wajid,SO,86.72,overcast clouds,3.8093,43.2471,
23,Kutum,SD,78.04,broken clouds,14.2000,24.6667,
28,Bengkulu,ID,75.47,overcast clouds,-3.8004,102.2655,
30,Ewa Beach,US,76.95,clear sky,21.3156,-158.0072,
31,Pontianak,ID,75.52,overcast clouds,-0.0333,109.3333,
34,Cassilandia,BR,89.85,clear sky,-19.1133,-51.7342,
35,Sao Sebastiao Do Paraiso,BR,82.99,clear sky,-20.9169,-46.9914,


In [69]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params= params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,'Hotel Name'] = hotels['results'][0]['name']
    except(IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [74]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df['Hotel Name'] != '']
len(clean_hotel_df)

232

In [77]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [89]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [90]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))